In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from scipy import stats

In [2]:
meld_path = '../../data/MELD.Raw'
data = pd.read_csv(os.path.join(meld_path, 'test_sent_emo.csv'))
data.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378"
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385"
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520"
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146"
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833"


In [3]:
# vid = cv2.VideoCapture('../dev_meld/dev_splits_complete/dia0_utt0.mp4')

# vid.get(cv2.CAP_PROP_FPS) # 24 fps

In [4]:
au_info = pd.read_csv('au_info.csv')
au_info.head()

,AU,FACS Name,Muscles,FACS Category,Related Expression,Models,Notes
0,AU01,Inner Brow Raiser,Frontalis (medial),main,"sadness, surprise, fear","svm, xgb, Py-Feat viz",NaN
1,AU02,Outer Brow Raiser,Frontalis (lateral),main,"surprise, fear","svm, xgb, Py-Feat viz",NaN
2,AU03,Inner corner Brow Tightener,"Procerus, Depressor Supercilii, Corrugator Sup...",extended (Baby FACS),NaN,none,Only in babies! Analogue of AU4 in adults
3,AU04,Brow Lowerer,"Procerus, Depressor Supercilii, Corrugator Sup...",main,"sadness, fear, anger","svm, xgb, Py-Feat viz",NaN
4,AU05,Upper Lid Raiser,"Levator Palpebrae Superioris, Superior Tarsal ...",main,"surprise, fear, anger","svm, xgb, Py-Feat viz",NaN


In [5]:
au2text = {row['AU']: row["FACS Name"] for i, row in au_info.iterrows()}
au2text['AU01']

'Inner Brow Raiser'

In [6]:
emo_au = { # https://en.wikipedia.org/wiki/Facial_Action_Coding_System
    "happy": ["AU06", "AU12"],
    "sadness": ["AU01", "AU04", "AU15"],
    "surprise": [ "AU01", "AU02", "AU05", "AU26"],
    "fear": ["AU01", "AU02", "AU04", "AU05", "AU07", "AU20", "AU26"],
    "anger": [ "AU04", "AU05", "AU07", "AU23"],
    "disgust": ["AU09", "AU15", "AU17"]
}

In [7]:
from feat import Detector

detector = Detector(device='cuda')
detector

/home/mkoltunov/anaconda3/envs/eacl_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Detector(face_model=img2pose, landmark_model=mobilefacenet, au_model=xgb, emotion_model=resmasknet, facepose_model=img2pose, identity_model=facenet)

In [8]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [9]:
# video_prediction = detector.detect_video(
#     '../dev_meld/dev_splits_complete/dia0_utt1.mp4', 
#     skip_frames=24, 
#     face_detection_threshold=0.95, 
#     progress_bar=False
# )
# video_prediction

In [10]:
# video_prediction.Identity

In [ ]:
# video_prediction[video_prediction['Identity'] == 'Person_0']

,FaceRectX,FaceRectY,FaceRectWidth,FaceRectHeight,FaceScore,x_0,x_1,x_2,x_3,x_4,...,Identity_506,Identity_507,Identity_508,Identity_509,Identity_510,Identity_511,Identity_512,input,frame,approx_time
frame,,,,,,,,,,,,,,,,,,,,,
0,856.149909,261.789694,156.852238,205.793272,0.9974,851.791738,854.320302,858.831132,865.492809,875.794341,...,0.004615,0.019514,-0.006874,0.012409,0.014877,-0.058626,-0.017865,../dev_meld/dev_splits_complete/dia0_utt1.mp4,0,00:00


In [ ]:
# video_prediction.emotions.columns

Index(['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise',
       'neutral'],
      dtype='object')

In [ ]:
# np.array(video_prediction.emotions).argmax(axis=1)

array([5, 4, 5], dtype=int64)

In [ ]:
# video_prediction.aus[['AU01', 'AU02']]

,AU01,AU02,sum
frame,,,
0,0.441943,0.318931,0.760874
0,0.763370,0.610041,1.373411
24,0.662878,0.253876,0.916754


In [ ]:
# video_prediction.emotions

,anger,disgust,fear,happiness,sadness,surprise,neutral
frame,,,,,,,
0,0.170836,0.001068,0.004450,0.001539,0.006734,0.755128,0.060246
0,0.022024,0.014874,0.023132,0.000953,0.565973,0.114586,0.258457
24,0.013036,0.001891,0.025513,0.002714,0.266660,0.468158,0.222028


In [11]:
emo_labels = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']

In [12]:
au_descs = []

for i, row in tqdm(data.iterrows()):
    try:
        try:
            video_path = os.path.join(meld_path, "output_repeated_splits_test", f"dia{row['Dialogue_ID']}_utt{row['Utterance_ID']}.mp4")
            video_prediction = detector.detect(
                video_path,
                data_type='video', 
                skip_frames=24,
                face_detection_threshold=0.95, 
                progress_bar=True,
                batch_size=8
            )
            spk_preds = video_prediction[video_prediction['Identity'] == 'Person_0']
            pseudo_label = stats.mode(np.array(spk_preds.emotions[emo_labels[:-1]]).argmax(axis=1))[0]
            pseudo_emo = emo_labels[pseudo_label]
            aus = emo_au[pseudo_emo]
            desc = ""
            for au in aus:
                desc += f"{au2text[au]}, "
            desc = desc[:-2]
            # aus_scores = video_prediction.aus[aus]
            # emo_peak_fr = spk_preds.index[np.array(aus_scores).sum(axis=1).argmax()]
            
        except:
            video_path = os.path.join(meld_path, "output_repeated_splits_test", f"final_videos_testdia{row['Dialogue_ID']}_utt{row['Utterance_ID']}.mp4")
            video_prediction = detector.detect(
                video_path,
                data_type='video', 
                skip_frames=24,
                face_detection_threshold=0.95, 
                progress_bar=True,
                batch_size=8
            )
            spk_preds = video_prediction[video_prediction['Identity'] == 'Person_0']
            pseudo_label = stats.mode(np.array(spk_preds.emotions[emo_labels[:-1]]).argmax(axis=1))[0]
            pseudo_emo = emo_labels[pseudo_label]
            aus = emo_au[pseudo_emo]
            desc = ""
            for au in aus:
                desc += f"{au2text[au]}, "
            desc = desc[:-2]
    except:
        desc = "Nan"
    au_descs.append(desc)

0it [00:00, ?it/s]
100%|██████████| 1/1 [00:01<00:00,  1.12s/it]
1it [00:01,  1.33s/it]
100%|██████████| 1/1 [00:03<00:00,  3.17s/it]
2it [00:04,  2.48s/it]
100%|██████████| 1/1 [00:00<00:00,  1.92it/s]
3it [00:05,  1.64s/it]
100%|██████████| 1/1 [00:00<00:00,  2.00it/s]
4it [00:05,  1.24s/it]
100%|██████████| 1/1 [00:00<00:00,  3.54it/s]
5it [00:06,  1.03it/s]
100%|██████████| 1/1 [00:01<00:00,  1.14s/it]
6it [00:07,  1.07s/it]
100%|██████████| 1/1 [00:00<00:00,  1.31it/s]
7it [00:08,  1.01s/it]
100%|██████████| 1/1 [00:00<00:00,  1.33it/s]
8it [00:09,  1.01it/s]
100%|██████████| 1/1 [00:02<00:00,  2.20s/it]
9it [00:11,  1.41s/it]
100%|██████████| 1/1 [00:00<00:00,  1.31it/s]
10it [00:12,  1.25s/it]
100%|██████████| 1/1 [00:00<00:00,  2.29it/s]
11it [00:13,  1.06s/it]
100%|██████████| 1/1 [00:00<00:00,  2.29it/s]
12it [00:13,  1.10it/s]
100%|██████████| 1/1 [00:00<00:00,  1.35it/s]
13it [00:14,  1.12it/s]
100%|██████████| 1/1 [00:01<00:00,  1.22s/it]
14it [00:16,  1.03s/it]
100%|█████

In [13]:
mod_data = pd.read_csv('../data/MELD/modified3_test_data.csv')
mod_data.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,audio_caption,visual_caption,Bio
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378",Neutral,The image shows a man and woman standing in a ...,"Mark appears curious and observant, likely pay..."
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385",The speaker sounds happy.,The image shows a man and woman standing in a ...,Rachel appears to be casual and humorous. Her ...
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520",The speaker sounds like a woman.,The image shows a man and woman sitting at a t...,Rachel appears to be casual and humorous. Her ...
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146",The speaker sounds angry.,The image shows a woman laying in a hospital b...,The given conversation does not include any ut...
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833","The speaker sounds male, aged between 16-25 ye...",The image shows a woman laying in a hospital b...,The given conversation does not include any ut...


In [14]:
mod_data['aus'] = au_descs
mod_data.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,audio_caption,visual_caption,Bio,aus
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378",Neutral,The image shows a man and woman standing in a ...,"Mark appears curious and observant, likely pay...","Inner Brow Raiser, Brow Lowerer, Lip Corner De..."
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385",The speaker sounds happy.,The image shows a man and woman standing in a ...,Rachel appears to be casual and humorous. Her ...,"Brow Lowerer, Upper Lid Raiser, Lid Tightener,..."
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520",The speaker sounds like a woman.,The image shows a man and woman sitting at a t...,Rachel appears to be casual and humorous. Her ...,"Brow Lowerer, Upper Lid Raiser, Lid Tightener,..."
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146",The speaker sounds angry.,The image shows a woman laying in a hospital b...,The given conversation does not include any ut...,"Brow Lowerer, Upper Lid Raiser, Lid Tightener,..."
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833","The speaker sounds male, aged between 16-25 ye...",The image shows a woman laying in a hospital b...,The given conversation does not include any ut...,"Brow Lowerer, Upper Lid Raiser, Lid Tightener,..."


In [15]:
mod_data.to_csv('../data/MELD/modified4_test_data.csv')